In [14]:
import os, sys
from PIL import Image
from PIL.ExifTags import TAGS
chemin_image='./cattt'
for fichier in os.listdir(chemin_image):
    if fichier.endswith('.jpg') :
        filepath = os.path.join(chemin_image, fichier)
        try:
            imgfile = Image.open(filepath)
            # Recupérer les métadonnées EXIF
            exif_data = imgfile._getexif()
            if exif_data:
                print("Métadonnées pour l'image {filename}:")
            for tag,value in exif_data.items():
                if tag in TAGS:
                    print(f"{TAGS[tag]}: {value}")
                else:
                    print(f"Aucune métadonné EXIF disponible pour l'image {imgfile}.")
        except (OSError, AttributeError):
                print(f"Impossible de traiter l'image {imgfile}.")

 


if exif_data: # s'il existe des informations EXIF
    for tag, value in exif_data.items():
        if tag in TAGS:
            print(TAGS[tag], value)


Impossible de traiter l'image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x417 at 0x10E0C3BD0>.
Impossible de traiter l'image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=584x1024 at 0x11E7DBF10>.
Impossible de traiter l'image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x681 at 0x11CC1F490>.
Impossible de traiter l'image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500 at 0x11DEBA190>.
Impossible de traiter l'image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x375 at 0x11DEBB690>.
Impossible de traiter l'image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x400 at 0x11CC0CCD0>.
Impossible de traiter l'image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=375x500 at 0x11E659CD0>.
Impossible de traiter l'image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x333 at 0x11DEBB690>.
Impossible de traiter l'image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x375 at 0x10E0C3BD0>.
Impossib

Visualisation des données 

In [3]:
from pandas import json_normalize
import pandas as pd
import json
import matplotlib.pyplot as plot

data = json.load(open("../datamining_project/metadata.json"))
dataframe = json_normalize(data)
grouped = dataframe.groupby("year").count()
plot.plot(grouped)
plot.show()

KeyError: 'year'